# HOME CREDIT DEFAULT RISK 

## Business Objective

Provide retail banks with risk evaluation models to predict the likelihood of loan default using applicant data.

## Data Loading and Overview

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from utils import read_csv_file, profile_missing, clean_application_data, clean_all_home_credit_tables
from IPython.display import display

## Per Table 

In [3]:
pd.set_option('display.max_colwidth', None)


### Application Train

In [4]:
df_app = pd.read_csv('application_train.csv')
col_desc = read_csv_file('HomeCredit_columns_description.csv', encoding='ISO-8859-1')

# === 2. Clean and filter column descriptions ===
# Keep only relevant columns
col_desc_cleaned = col_desc[['Table', 'Row', 'Description']].copy()

# Filter for application train/test only
desc_app = col_desc_cleaned[col_desc_cleaned['Table'].astype(str) == 'application_{train|test}.csv'].copy()

# === 3. Profile missing values ===
missing_summary = df_app.isnull().sum().to_frame(name='MissingCount')
missing_summary['MissingRatio'] = missing_summary['MissingCount'] / len(df_app)
missing_summary = missing_summary[missing_summary['MissingCount'] > 0].reset_index()
missing_summary.columns = ['Row', 'MissingCount', 'MissingRatio']

# === 4. Normalize for safe merging ===
desc_app['Row_clean'] = desc_app['Row'].astype(str).str.strip().str.upper()
missing_summary['Row_clean'] = missing_summary['Row'].astype(str).str.strip().str.upper()

# === 5. Merge ===
missing_merged = pd.merge(
    missing_summary,
    desc_app[['Row_clean', 'Description']],
    on='Row_clean',
    how='left'
)

# === 6. Show output ===
missing_merged['Row'] = missing_summary['Row']
display(missing_merged[['Row', 'MissingRatio', 'Description']].head(20))

Loaded 'HomeCredit_columns_description.csv' — shape: (219, 5)


,Row,MissingRatio,Description
0,AMT_ANNUITY,0.000039,Loan annuity
1,AMT_GOODS_PRICE,0.000904,For consumer loans it is the price of the goods for which the loan is given
2,NAME_TYPE_SUITE,0.004201,Who was accompanying client when he was applying for the loan
3,OWN_CAR_AGE,0.659908,Age of client's car
4,OCCUPATION_TYPE,0.313455,What kind of occupation does the client have
5,CNT_FAM_MEMBERS,0.000007,How many family members does client have
6,EXT_SOURCE_1,0.563811,Normalized score from external data source
7,EXT_SOURCE_2,0.002146,Normalized score from external data source
8,EXT_SOURCE_3,0.198253,Normalized score from external data source
9,APARTMENTS_AVG,0.507497,"Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor"


### Bureau 

In [5]:
df_bureau = read_csv_file('bureau.csv')
col_desc = read_csv_file('HomeCredit_columns_description.csv', encoding='ISO-8859-1')

# === 2. Clean column descriptions ===
col_desc_cleaned = col_desc[['Table', 'Row', 'Description']].copy()

# Filter for bureau.csv
desc_bureau = col_desc_cleaned[col_desc_cleaned['Table'].astype(str) == 'bureau.csv'].copy()

# === 3. Profile missing values ===
missing_summary = df_bureau.isnull().sum().to_frame(name='MissingCount')
missing_summary['MissingRatio'] = missing_summary['MissingCount'] / len(df_bureau)
missing_summary = missing_summary[missing_summary['MissingCount'] > 0].reset_index()
missing_summary.columns = ['Row', 'MissingCount', 'MissingRatio']

# === 4. Normalize for merging ===
desc_bureau['Row_clean'] = desc_bureau['Row'].astype(str).str.strip().str.upper()
missing_summary['Row_clean'] = missing_summary['Row'].astype(str).str.strip().str.upper()

# === 5. Merge ===
missing_merged = pd.merge(
    missing_summary,
    desc_bureau[['Row_clean', 'Description']],
    on='Row_clean',
    how='left'
)

# === 6. Display ===
missing_merged['Row'] = missing_summary['Row']
display(missing_merged[['Row', 'MissingRatio', 'Description']].head(20))

Loaded 'bureau.csv' — shape: (1716428, 17)
Loaded 'HomeCredit_columns_description.csv' — shape: (219, 5)


,Row,MissingRatio,Description
0,DAYS_CREDIT_ENDDATE,0.061496,Remaining duration of CB credit (in days) at the time of application in Home Credit
1,DAYS_ENDDATE_FACT,0.369170,Days since CB credit ended at the time of application in Home Credit (only for closed credit)
2,AMT_CREDIT_MAX_OVERDUE,0.655133,Maximal amount overdue on the Credit Bureau credit so far (at application date of loan in our sample)
3,AMT_CREDIT_SUM,0.000008,Current credit amount for the Credit Bureau credit
4,AMT_CREDIT_SUM_DEBT,0.150119,Current debt on Credit Bureau credit
5,AMT_CREDIT_SUM_LIMIT,0.344774,Current credit limit of credit card reported in Credit Bureau
6,AMT_ANNUITY,0.714735,Annuity of the Credit Bureau credit


### Bureau Balance

In [6]:
df_bureau_bal = read_csv_file('bureau_balance.csv')
col_desc = read_csv_file('HomeCredit_columns_description.csv', encoding='ISO-8859-1')

# === 2. Clean column descriptions ===
col_desc_cleaned = col_desc[['Table', 'Row', 'Description']].copy()

# Filter for bureau.csv
desc_bureau_bal = col_desc_cleaned[col_desc_cleaned['Table'].astype(str) == 'bureau_balance.csv'].copy()

# === 3. Profile missing values ===
missing_summary = df_bureau_bal.isnull().sum().to_frame(name='MissingCount')
missing_summary['MissingRatio'] = missing_summary['MissingCount'] / len(df_bureau_bal)
missing_summary = missing_summary[missing_summary['MissingCount'] > 0].reset_index()
missing_summary.columns = ['Row', 'MissingCount', 'MissingRatio']

# === 4. Normalize for merging ===
desc_bureau_bal['Row_clean'] = desc_bureau_bal['Row'].astype(str).str.strip().str.upper()
missing_summary['Row_clean'] = missing_summary['Row'].astype(str).str.strip().str.upper()

# === 5. Merge ===
missing_merged = pd.merge(
    missing_summary,
    desc_bureau_bal[['Row_clean', 'Description']],
    on='Row_clean',
    how='left'
)

# === 6. Display ===
missing_merged['Row'] = missing_summary['Row']
display(missing_merged[['Row', 'MissingRatio', 'Description']].head(20))

Loaded 'bureau_balance.csv' — shape: (27299925, 3)
Loaded 'HomeCredit_columns_description.csv' — shape: (219, 5)


,Row,MissingRatio,Description


### Previous Application

In [7]:
# Load files
df_prev = pd.read_csv('previous_application.csv')
col_desc = pd.read_csv('HomeCredit_columns_description.csv', encoding='ISO-8859-1')

# Clean column description
col_desc_cleaned = col_desc[['Table', 'Row', 'Description']].copy()
desc_prev = col_desc_cleaned[col_desc_cleaned['Table'].astype(str) == 'previous_application.csv'].copy()

# Profile missing values
missing_summary = df_prev.isnull().sum().to_frame(name='MissingCount')
missing_summary['MissingRatio'] = missing_summary['MissingCount'] / len(df_prev)
missing_summary = missing_summary[missing_summary['MissingCount'] > 0].reset_index()
missing_summary.columns = ['Row', 'MissingCount', 'MissingRatio']

# Normalize keys for merge
desc_prev['Row_clean'] = desc_prev['Row'].astype(str).str.strip().str.upper()
missing_summary['Row_clean'] = missing_summary['Row'].astype(str).str.strip().str.upper()

# Merge
missing_merged = pd.merge(
    missing_summary,
    desc_prev[['Row_clean', 'Description']],
    on='Row_clean',
    how='left'
)

# Restore original row name
missing_merged['Row'] = missing_summary['Row']

# Display result
display(missing_merged[['Row', 'MissingRatio', 'Description']])


,Row,MissingRatio,Description
0,AMT_ANNUITY,2.228667e-01,Annuity of previous application
1,AMT_CREDIT,5.987257e-07,"Final credit amount on the previous application. This differs from AMT_APPLICATION in a way that the AMT_APPLICATION is the amount for which the client initially applied for, but during our approval process he could have received different amount - AMT_CREDIT"
2,AMT_DOWN_PAYMENT,5.363648e-01,Down payment on the previous application
3,AMT_GOODS_PRICE,2.308177e-01,Goods price of good that client asked for (if applicable) on the previous application
4,RATE_DOWN_PAYMENT,5.363648e-01,Down payment rate normalized on previous credit
5,RATE_INTEREST_PRIMARY,9.964370e-01,Interest rate normalized on previous credit
6,RATE_INTEREST_PRIVILEGED,9.964370e-01,Interest rate normalized on previous credit
7,NAME_TYPE_SUITE,4.911975e-01,Who accompanied client when applying for the previous application
8,CNT_PAYMENT,2.228637e-01,Term of previous credit at application of the previous application
9,PRODUCT_COMBINATION,2.071591e-04,Detailed product combination of the previous application


### Credit Card Balance

In [8]:
# Load files
df_cc = pd.read_csv('credit_card_balance.csv')
col_desc = pd.read_csv('HomeCredit_columns_description.csv', encoding='ISO-8859-1')

# Clean column description
col_desc_cleaned = col_desc[['Table', 'Row', 'Description']].copy()
desc_cc = col_desc_cleaned[col_desc_cleaned['Table'].astype(str) == 'credit_card_balance.csv'].copy()

# Profile missing values
missing_summary = df_cc.isnull().sum().to_frame(name='MissingCount')
missing_summary['MissingRatio'] = missing_summary['MissingCount'] / len(df_cc)
missing_summary = missing_summary[missing_summary['MissingCount'] > 0].reset_index()
missing_summary.columns = ['Row', 'MissingCount', 'MissingRatio']

# Normalize keys for merge
desc_cc['Row_clean'] = desc_cc['Row'].astype(str).str.strip().str.upper()
missing_summary['Row_clean'] = missing_summary['Row'].astype(str).str.strip().str.upper()

# Merge
missing_merged = pd.merge(
    missing_summary,
    desc_cc[['Row_clean', 'Description']],
    on='Row_clean',
    how='left'
)

# Restore original row name
missing_merged['Row'] = missing_summary['Row']

# Display result
display(missing_merged[['Row', 'MissingRatio', 'Description']])


,Row,MissingRatio,Description
0,AMT_DRAWINGS_ATM_CURRENT,0.195249,Amount drawing at ATM during the month of the previous credit
1,AMT_DRAWINGS_OTHER_CURRENT,0.195249,Amount of other drawings during the month of the previous credit
2,AMT_DRAWINGS_POS_CURRENT,0.195249,Amount drawing or buying goods during the month of the previous credit
3,AMT_INST_MIN_REGULARITY,0.079482,Minimal installment for this month of the previous credit
4,AMT_PAYMENT_CURRENT,0.199981,How much did the client pay during the month on the previous credit
5,CNT_DRAWINGS_ATM_CURRENT,0.195249,Number of drawings at ATM during this month on the previous credit
6,CNT_DRAWINGS_OTHER_CURRENT,0.195249,Number of other drawings during this month on the previous credit
7,CNT_DRAWINGS_POS_CURRENT,0.195249,Number of drawings for goods during this month on the previous credit
8,CNT_INSTALMENT_MATURE_CUM,0.079482,Number of paid installments on the previous credit


### POS Cash Balance

In [9]:
# Load files
df_pos = pd.read_csv('POS_CASH_balance.csv')
col_desc = pd.read_csv('HomeCredit_columns_description.csv', encoding='ISO-8859-1')

# Clean column description
col_desc_cleaned = col_desc[['Table', 'Row', 'Description']].copy()
desc_pos = col_desc_cleaned[col_desc_cleaned['Table'].astype(str) == 'POS_CASH_balance.csv'].copy()

# Profile missing values
missing_summary = df_pos.isnull().sum().to_frame(name='MissingCount')
missing_summary['MissingRatio'] = missing_summary['MissingCount'] / len(df_pos)
missing_summary = missing_summary[missing_summary['MissingCount'] > 0].reset_index()
missing_summary.columns = ['Row', 'MissingCount', 'MissingRatio']

# Normalize keys for merge
desc_pos['Row_clean'] = desc_pos['Row'].astype(str).str.strip().str.upper()
missing_summary['Row_clean'] = missing_summary['Row'].astype(str).str.strip().str.upper()

# Merge
missing_merged = pd.merge(
    missing_summary,
    desc_pos[['Row_clean', 'Description']],
    on='Row_clean',
    how='left'
)

# Restore original row name
missing_merged['Row'] = missing_summary['Row']

# Display result
display(missing_merged[['Row', 'MissingRatio', 'Description']])


,Row,MissingRatio,Description
0,CNT_INSTALMENT,0.002607,Term of previous credit (can change over time)
1,CNT_INSTALMENT_FUTURE,0.002608,Installments left to pay on the previous credit


### Installments Payments

In [10]:
# Load files
df_install_pay = pd.read_csv('installments_payments.csv')
col_desc = pd.read_csv('HomeCredit_columns_description.csv', encoding='ISO-8859-1')

# Clean column description
col_desc_cleaned = col_desc[['Table', 'Row', 'Description']].copy()
desc_install_pay = col_desc_cleaned[col_desc_cleaned['Table'].astype(str) == 'installments_payments.csv'].copy()

# Profile missing values
missing_summary = df_install_pay.isnull().sum().to_frame(name='MissingCount')
missing_summary['MissingRatio'] = missing_summary['MissingCount'] / len(df_install_pay)
missing_summary = missing_summary[missing_summary['MissingCount'] > 0].reset_index()
missing_summary.columns = ['Row', 'MissingCount', 'MissingRatio']

# Normalize keys for merge
desc_install_pay['Row_clean'] = desc_install_pay['Row'].astype(str).str.strip().str.upper()
missing_summary['Row_clean'] = missing_summary['Row'].astype(str).str.strip().str.upper()

# Merge
missing_merged = pd.merge(
    missing_summary,
    desc_install_pay[['Row_clean', 'Description']],
    on='Row_clean',
    how='left'
)

# Restore original row name
missing_merged['Row'] = missing_summary['Row']

# Display result
display(missing_merged[['Row', 'MissingRatio', 'Description']])


,Row,MissingRatio,Description
0,DAYS_ENTRY_PAYMENT,0.000214,When was the installments of previous credit paid actually (relative to application date of current loan)
1,AMT_PAYMENT,0.000214,What the client actually paid on previous credit on this installment


## Handliing Missing Values

In [11]:
cleaned_tables = clean_all_home_credit_tables()

cleaned_tables['application']
cleaned_tables['bureau']
cleaned_tables['installments']
...


Loaded 'application_train.csv' — shape: (307511, 122)
Loaded 'application_test.csv' — shape: (48744, 121)
Loaded 'bureau.csv' — shape: (1716428, 17)
Loaded 'bureau_balance.csv' — shape: (27299925, 3)
Loaded 'previous_application.csv' — shape: (1670214, 37)
Loaded 'credit_card_balance.csv' — shape: (3840312, 23)
Loaded 'POS_CASH_balance.csv' — shape: (10001358, 8)
Loaded 'installments_payments.csv' — shape: (13605401, 8)

 Cleaning application.csv

 Cleaning application_test.csv

 Cleaning bureau.csv

 Cleaning bureau_balance.csv

 Cleaning previous_application.csv

 Cleaning credit_card_balance.csv

 Cleaning pos_cash.csv

 Cleaning installments.csv


Ellipsis

### Saving Cleaned Tables

In [12]:
for name, df in cleaned_tables.items():
    print(f"Saving {name}_cleaned.csv ...")
    df.to_csv(f"{name}_cleaned.csv", index=False)


Saving application_cleaned.csv ...
Saving application_test_cleaned.csv ...
Saving bureau_cleaned.csv ...
Saving bureau_balance_cleaned.csv ...
Saving previous_application_cleaned.csv ...
Saving credit_card_balance_cleaned.csv ...
Saving pos_cash_cleaned.csv ...
Saving installments_cleaned.csv ...


## Feature Engineering, Auxillary Tables

### Bureau Feature Block

In [13]:

# Load only required columns from bureau_balance and bureau
df_bbal = pd.read_csv("bureau_balance.csv", usecols=["SK_ID_BUREAU", "MONTHS_BALANCE", "STATUS"])
df_bureau = pd.read_csv("bureau_cleaned.csv", usecols=["SK_ID_BUREAU", "SK_ID_CURR"])

# Join to get SK_ID_CURR in bureau_balance
df_joined = df_bbal.merge(df_bureau, on="SK_ID_BUREAU", how="left")

# One-hot encode STATUS
status_dummies = pd.get_dummies(df_joined["STATUS"], prefix="STATUS")
df_joined = pd.concat([df_joined, status_dummies], axis=1)

# Aggregate temporal features by SK_ID_CURR
agg_dict = {
    "MONTHS_BALANCE": ["min", "max", "count"],
    "STATUS_0": "sum",
    "STATUS_1": "sum",
    "STATUS_2": "sum",
    "STATUS_3": "sum",
    "STATUS_4": "sum",
    "STATUS_5": "sum",
    "STATUS_C": "sum",
    "STATUS_X": "sum"
}
bureau_time_features = df_joined.groupby("SK_ID_CURR").agg(agg_dict)

# Flatten column names
bureau_time_features.columns = ["bureau_" + "_".join(col) for col in bureau_time_features.columns]
bureau_time_features.reset_index(inplace=True)

# Save to CSV
bureau_time_features.to_csv("bureau_time_features.csv", index=False)

# Optional: check shape
print(bureau_time_features.shape)


(134542, 12)


### Bureau Balance Temporal Feature Engineering
I engineered time-aware features from `bureau_balance.csv`, joined to the bureau table using `SK_ID_BUREAU`, and aggregated by `SK_ID_CURR`.

Key steps:

- Loaded only the relevant columns: `SK_ID_BUREAU`, `MONTHS_BALANCE`, `STATUS`  

- Merged to get `SK_ID_CURR` from the bureau table  

- One-hot encoded the monthly STATUS flags:
    - STATUS values meaning: 
    
        - 0 to 5: number of months past due (DPD)

        - C: account was closed that month 
        
        - X: no DPD / account not active that month

- Aggregated features per client (SK_ID_CURR): 

- `MONTHS_BALANCE`: min, max, and count → reflects credit age, recency, and duration

- `STATUS_`*: sum of each flag → counts of how many times the account was in each state 


Final output (bureau_time_features.csv) has 12 columns and preserves temporal credit risk dynamics such as delinquency history and duration of bureau activity.



### Previous Application

In [14]:

# Load cleaned data
df_prev = pd.read_csv("previous_application_cleaned.csv")

# Define columns
id_col = 'SK_ID_CURR'

# One-hot encode application status
status_dummies = pd.get_dummies(df_prev['NAME_CONTRACT_STATUS'], prefix='APP_STATUS')
df_prev = pd.concat([df_prev, status_dummies], axis=1)

# Aggregation logic
agg_dict = {
    'SK_ID_PREV': 'count',  # total number of previous applications
    'DAYS_DECISION': ['min', 'max'],  # recency of applications
    'AMT_APPLICATION': ['mean'],      # avg amount requested
    'AMT_CREDIT': ['mean'],           # avg amount granted
    'APP_STATUS_Approved': 'sum',
    'APP_STATUS_Canceled': 'sum',
    'APP_STATUS_Refused': 'sum',
    'APP_STATUS_Unused offer': 'sum'
}

# Group and aggregate
prev_time_features = df_prev.groupby(id_col).agg(agg_dict)

# Flatten column names
prev_time_features.columns = ['prev_' + '_'.join(col).strip() if isinstance(col, tuple) else f'prev_{col}'
                              for col in prev_time_features.columns]

# Reset index and save
prev_time_features.reset_index(inplace=True)
prev_time_features.to_csv("previous_application_features.csv", index=False)

# Optional: shape check
print("previous_application_features.csv ", prev_time_features.shape)


previous_application_features.csv  (338857, 10)


### Aggregating `previous_application.csv` — Preserving Time & Application Patterns

This block summarizes key historical behaviors from the previous_application table while avoiding high-dimensional outputs:

- Goal: Capture temporal and categorical trends in previous loans/applications per client (`SK_ID_CURR`) without exploding the feature space. 

- Steps Taken:
    - One-hot encoded `NAME_CONTRACT_STATUS` to count the number of:
        - Approved

        - Refused

        - Canceled

        - Unused Offers

- Aggregated `DAYS_DECISION` to extract application recency (min, max). 

- Averaged financial variables: 

    - `AMT_APPLICATION` – amount requested

    - `AMT_CREDIT` – amount granted 

- Counted total previous applications using SK_ID_PREV. 

- Result: A lean, interpretable temporal feature table with ~10–12 columns per client

### Credit Card Balance

In [15]:
# Load cleaned credit card data
df_cc = pd.read_csv("credit_card_balance_cleaned.csv")

# Optional: reduce to necessary columns for speed
df_cc = df_cc[[
    "SK_ID_CURR", "MONTHS_BALANCE",
    "AMT_BALANCE", "AMT_CREDIT_LIMIT_ACTUAL",
    "AMT_DRAWINGS_CURRENT", "CNT_INSTALMENT_MATURE_CUM"
]]

# Group and aggregate
cc_agg = df_cc.groupby("SK_ID_CURR").agg({
    "MONTHS_BALANCE": ["min", "max", "count"],
    "AMT_BALANCE": "mean",
    "AMT_CREDIT_LIMIT_ACTUAL": "mean",
    "AMT_DRAWINGS_CURRENT": ["sum", "mean"],
    "CNT_INSTALMENT_MATURE_CUM": "mean"
})

# Flatten column names
cc_agg.columns = [f"cc_{'_'.join(col)}" for col in cc_agg.columns]
cc_agg.reset_index(inplace=True)

# Save to disk
cc_agg.to_csv("credit_card_balance_features.csv", index=False)
print(cc_agg.shape)


(103558, 9)


### Feature Engineering: `credit_card_balance.csv`

This block aggregates temporal and behavioral patterns from the `credit_card_balance` table per client (`SK_ID_CURR`) while preserving essential time-based information.

#### Features created:

- `cc_MONTHS_BALANCE_min`: The most recent statement month (e.g., -1 is most recent).

- `cc_MONTHS_BALANCE_max`: The earliest statement available (i.e., longest time on file).

- `cc_MONTHS_BALANCE_count`: Number of monthly statements (proxy for duration of card activity).

- `cc_AMT_BALANCE_mean`: Average monthly balance across all months.

- `cc_AMT_CREDIT_LIMIT_ACTUAL_mean`: Average credit limit (client capacity).

- `cc_AMT_DRAWINGS_CURRENT_sum`: Total amount drawn from credit card (indicative of usage).

- `cc_AMT_DRAWINGS_CURRENT_mean`: Average monthly draw amount.

- `cc_CNT_INSTALMENT_MATURE_CUM_mean`: Average number of expected installments (credit type indicator).

These features help capture **recency**, **activity intensity**, and **account longevity**, all of which are predictive in credit risk modeling.


### POS Cash

In [16]:
# Load cleaned POS_CASH_balance
df_pos = pd.read_csv("pos_cash_cleaned.csv")

# Filter only needed columns for compact memory usage
df_pos = df_pos[[
    "SK_ID_CURR", "MONTHS_BALANCE",
    "CNT_INSTALMENT", "CNT_INSTALMENT_FUTURE"
]]

# Aggregate by SK_ID_CURR
pos_agg = df_pos.groupby("SK_ID_CURR").agg({
    "MONTHS_BALANCE": ["min", "max", "count"],
    "CNT_INSTALMENT": ["mean", "max"],
    "CNT_INSTALMENT_FUTURE": ["mean", "max"]
})

# Flatten column names
pos_agg.columns = [f"pos_{'_'.join(col)}" for col in pos_agg.columns]
pos_agg.reset_index(inplace=True)

# Save
pos_agg.to_csv("pos_cash_features.csv", index=False)
print(pos_agg.shape)


(337252, 8)


### Feature Engineering: `pos_cash_cleaned.csv`

This block summarizes a client's point-of-sale and installment loan performance over time by aggregating records per `SK_ID_CURR`.

#### Features created:

- `pos_MONTHS_BALANCE_min`: Most recent POS loan update (e.g., -1 = current). 

- `pos_MONTHS_BALANCE_max`: Longest historical record available. 

- `pos_MONTHS_BALANCE_count`: Total POS loan statements — proxy for activity duration. 

- `pos_CNT_INSTALMENT_mean` / `max`: Typical and peak number of installments (credit scale). 

- `pos_CNT_INSTALMENT_FUTURE_mean` / `max`: Expected future repayments — proxy for outstanding obligations.

These time-aware features capture **loan frequency**, **repayment structure**, and **credit longevity**, all useful in default prediction.


### Installments Payments

In [17]:
# Load cleaned installments data
df_inst = pd.read_csv("installments_cleaned.csv")

# Create derived features
df_inst['PAYMENT_DELAY'] = df_inst['DAYS_ENTRY_PAYMENT'] - df_inst['DAYS_INSTALMENT']
df_inst['PAYMENT_DIFF'] = df_inst['AMT_PAYMENT'] - df_inst['AMT_INSTALMENT']

# Aggregate per client
inst_agg = df_inst.groupby("SK_ID_CURR").agg({
    'DAYS_INSTALMENT': ['min', 'max'],
    'PAYMENT_DELAY': ['mean', 'max', 'min'],
    'PAYMENT_DIFF': ['mean', 'min', 'max'],
    'AMT_INSTALMENT': 'mean',
    'AMT_PAYMENT': 'mean',
    'SK_ID_PREV': 'count'
})

# Flatten column names
inst_agg.columns = [f"inst_{'_'.join(col)}" for col in inst_agg.columns]
inst_agg.reset_index(inplace=True)

# Save
inst_agg.to_csv("installments_features.csv", index=False)
print(inst_agg.shape)


(339587, 12)


### Feature Engineering: `installments_payments.csv`

This block creates repayment behavior features from the client's installment loan history, grouped by `SK_ID_CURR`.

#### Features created:

- `inst_DAYS_INSTALMENT_min` / `max`: Earliest and latest scheduled payment due dates. 

- `inst_PAYMENT_DELAY_mean` / `min` / `max`: Average days early or late payment was made (`DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT`). 

- `inst_PAYMENT_DIFF_mean` / `min` / `max`: Difference between amount paid and amount due (`AMT_PAYMENT - AMT_INSTALMENT`), highlighting under/overpayments. 

- `inst_AMT_INSTALMENT_mean`: Typical scheduled payment amount. 

- `inst_AMT_PAYMENT_mean`: Typical actual payment amount. 

- `inst_SK_ID_PREV_count`: Total number of installment payments made. 

These features reflect **timing**, **compliance**, and **repayment behavior**, which are strong indicators of creditworthiness.


### Final Merge

In [18]:
# Load base application table
df_final = pd.read_csv("application_cleaned.csv")

# List of engineered feature files to merge
feature_files = [
    "bureau_time_features.csv",
    "previous_application_features.csv",
    "credit_card_balance_features.csv",
    "pos_cash_features.csv",
    "installments_features.csv"
]

# Sequentially merge each feature table
for file in feature_files:
    print(f" Merging {file} ...")
    df_feat = pd.read_csv(file)
    df_final = df_final.merge(df_feat, on="SK_ID_CURR", how="left")

# Save final merged dataset
df_final.to_csv("train_final.csv", index=False)
print(" train_final.csv saved — shape:", df_final.shape)


 Merging bureau_time_features.csv ...
 Merging previous_application_features.csv ...
 Merging credit_card_balance_features.csv ...
 Merging pos_cash_features.csv ...
 Merging installments_features.csv ...
 train_final.csv saved — shape: (307511, 272)


In [19]:
# Load the full merged dataset
df_final = pd.read_csv("train_final.csv")

# === 1. DROP excess FLAG_* columns except important ones ===
keep_flags = [
    'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
    'FLAG_MOBIL', 'FLAG_EMP_PHONE',
    'FLAG_WORK_PHONE', 'FLAG_PHONE',
    'FLAG_EMAIL'
]

flag_cols = [col for col in df_final.columns if col.startswith("FLAG_") and col not in keep_flags]
df_final.drop(columns=flag_cols, inplace=True)

# === 2. DROP _MISSING flag columns ===
missing_cols = [col for col in df_final.columns if col.endswith('_MISSING')]
df_final.drop(columns=missing_cols, inplace=True)

# === 3. DROP SK_ID_CURR (optional for modeling) ===
if 'SK_ID_CURR' in df_final.columns:
    df_final.drop(columns=['SK_ID_CURR'], inplace=True)

# === 4. Save compact version ===
df_final.to_csv("train_final_compact.csv", index=False)
print("Saved train_final_compact.csv — shape:", df_final.shape)


Saved train_final_compact.csv — shape: (307511, 146)


### Test Dataset Merge

In [20]:
# Load base application table
df_test = pd.read_csv("application_test_cleaned.csv")

# List of engineered feature files to merge
feature_files = [
    "bureau_time_features.csv",
    "previous_application_features.csv",
    "credit_card_balance_features.csv",
    "pos_cash_features.csv",
    "installments_features.csv"
]

# Sequentially merge each feature table
for file in feature_files:
    print(f" Merging {file} ...")
    df_feat = pd.read_csv(file)
    df_test = df_test.merge(df_feat, on="SK_ID_CURR", how="left")

# Save final merged dataset
df_test.to_csv("test_final.csv", index=False)
print(" test_final.csv saved — shape:", df_test.shape)


 Merging bureau_time_features.csv ...
 Merging previous_application_features.csv ...
 Merging credit_card_balance_features.csv ...
 Merging pos_cash_features.csv ...
 Merging installments_features.csv ...
 test_final.csv saved — shape: (48744, 271)


In [21]:
# Load the full merged dataset
df_test = pd.read_csv("test_final.csv")

# === 1. DROP excess FLAG_* columns except important ones ===
keep_flags = [
    'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
    'FLAG_MOBIL', 'FLAG_EMP_PHONE',
    'FLAG_WORK_PHONE', 'FLAG_PHONE',
    'FLAG_EMAIL'
]

flag_cols = [col for col in df_test.columns if col.startswith("FLAG_") and col not in keep_flags]
df_test.drop(columns=flag_cols, inplace=True)

# === 2. DROP _MISSING flag columns ===
missing_cols = [col for col in df_test.columns if col.endswith('_MISSING')]
df_test.drop(columns=missing_cols, inplace=True)

# === 4. Save compact version ===
df_test.to_csv("test_final_compact.csv", index=False)
print("Saved test_final_compact.csv — shape:", df_test.shape)


Saved test_final_compact.csv — shape: (48744, 146)
